# <font color="#003660">Applied Machine Learning for Text Analysis (M.184.5331)</font>


# <font color="#003660">Session 8: Retrieval-Augmented Generation</font>

# <font color="#003660">Reranking</font>

<center><br><img width=256 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/dag.png"/><br></center>

<p>

<div>
    <font color="#085986"><b>By the end of this lesson, you ...</b><br><br>
        ... will know how to implement reranking methods using langchain. <br>
        ... will know hot to implement langchain chains on your own.
    </font>
</div>
</p>

The following content is heavily inspired by the following excellent sources:

* [HuggingFace (2024): NLP Course](https://huggingface.co/learn/nlp-course/)
* [Huggingface (2024): Open-Source AI Cookbook](https://huggingface.co/learn/cookbook/index)
* [LangChain API Reference (2024)](https://python.langchain.com/api_reference/reference.html)
* [LangChain Docs (2024)](https://python.langchain.com/docs/introduction/)
* [LangChain AI (2024) Cookbook](https://github.com/langchain-ai/langchain/blob/master/cookbook/rewrite.ipynb?ref=blog.langchain.dev)

# Reranking

![](https://github.com/olivermueller/amlta-2024/blob/main/Session_08/imgs/rag_extensions.png?raw=true)

(Source: ([Wang et al., 2024](https://doi.org/10.18653/v1/2024.emnlp-main.981)))

There are multiple ways to improve RAG architectures as summarized by [Wang et al. (2024)](https://doi.org/10.18653/v1/2024.emnlp-main.981).

This lecture focuses on reranking.

In [ ]:
!pip install -U pymupdf4llm datasets transformers faiss-cpu sentence-transformers accelerate langchain langchain-community langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Su

In [ ]:
import os
import re
from tqdm.notebook import tqdm
import pymupdf4llm
import urllib

from IPython.display import display, Markdown

from transformers import AutoTokenizer
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain import hub
from langchain_huggingface import HuggingFacePipeline

from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank

DEVICE = "cuda"

In [ ]:
os.mkdir("documents")
os.mkdir("imgs")
os.mkdir("markdown_documents")
urllib.request.urlretrieve("https://raw.githubusercontent.com/olivermueller/amlta-2024/refs/heads/main/Session_08/documents/Game_of_Thrones.pdf", "documents/Game_of_Thrones.pdf")
urllib.request.urlretrieve("https://raw.githubusercontent.com/olivermueller/amlta-2024/refs/heads/main/Session_08/documents/How_I_Met_Your_Mother.pdf", "documents/How_I_Met_Your_Mother.pdf")
urllib.request.urlretrieve("https://raw.githubusercontent.com/olivermueller/amlta-2024/refs/heads/main/Session_08/markdown_documents/Game_of_Thrones.md", "markdown_documents/Game_of_Thrones.md")
urllib.request.urlretrieve("https://raw.githubusercontent.com/olivermueller/amlta-2024/refs/heads/main/Session_08/markdown_documents/How_I_Met_Your_Mother.md", "markdown_documents/How_I_Met_Your_Mother.md")

('markdown_documents/How_I_Met_Your_Mother.md',
 <http.client.HTTPMessage at 0x7eb32f3454e0>)

In [ ]:
RETRIEVER_NAME = "jinaai/jina-embeddings-v2-base-en"
GENERATOR_NAME = "Qwen/Qwen2.5-1.5B-Instruct"

# Loading Documents

In [ ]:
markdown_documents_path = "markdown_documents"

In [ ]:
def remove_markdown_links(text):
    """
    Removes Markdown links from the given text while keeping the link text.

    Args:
        text (str): The input Markdown text.

    Returns:
        str: The text with Markdown links removed.

    Yeah this was ChatGPT ;)
    """
    # Regex to match Markdown links [text](link)
    pattern = r'\[([^\]]+)\]\([^\)]+\)'
    # Replace the matched pattern with just the text inside the brackets
    cleaned_text = re.sub(pattern, r'\1', text)
    return cleaned_text

In [ ]:
markdown_documents = os.listdir(markdown_documents_path)

md_files = []

for markdown_document in markdown_documents:
    markdown_document_path = os.path.join(markdown_documents_path, markdown_document)
    with open(markdown_document_path) as file:
        md_files.append([markdown_document, remove_markdown_links(file.read())])

# Original Chain

In [ ]:
embedding_tokenizer = AutoTokenizer.from_pretrained(RETRIEVER_NAME, use_fast=False)
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    embedding_tokenizer,
    chunk_size=512,
    chunk_overlap=32,
    separators=[
        "# ",
        "## ",
        "### ",
        "#### ",
        "##### ",
        "###### ",
        "-----",
        "\n\n",
        "\n",
        " ",
    ],
    keep_separator=True
)
all_splits = text_splitter.create_documents(
    texts=[x[1] for x in md_files],
    metadatas=[{"source": x[0]} for x in md_files],
)

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

In [ ]:
retriever_model = HuggingFaceEmbeddings(
    model_name=RETRIEVER_NAME,
    model_kwargs={'device': DEVICE, "trust_remote_code": True},
    encode_kwargs={'normalize_embeddings': True}
)

db = FAISS.from_documents(
    all_splits,
    embedding=retriever_model
)
retriever = db.as_retriever()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/71.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-bert-implementation:
- configuration_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_bert.py:   0%|          | 0.00/97.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-bert-implementation:
- modeling_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/275M [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
llm = HuggingFacePipeline.from_model_id(
    model_id=GENERATOR_NAME,
    task="text-generation",
    pipeline_kwargs={"return_full_text": False}
)

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
prompt = hub.pull("rlm/rag-prompt")
print(prompt.messages[0].prompt.template)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:


/usr/local/lib/python3.10/dist-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


## New CrossEncoder Reranker

![](https://raw.githubusercontent.com/UKPLab/sentence-transformers/master/docs/img/Bi_vs_Cross-Encoder.png)

Source: ([SBERT.net (2024)](https://www.sbert.net/examples/applications/cross-encoder/README.html))

Cross-Encoder Rerankers output a numerical value between 0 and 1 and can be used to determine the similarity between two sentences (chunks). While they are not compute efficient on large-scale tasks such as clustering they can improve RAG systems by ranking the reranking the chunks by their query-similarity ([Reimers and Gurevych](https://doi.org/10.48550/arXiv.1908.10084)).

In [ ]:
model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")
compressor = CrossEncoderReranker(model=model, top_n=3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [ ]:
print([x.id for x in retriever.invoke("Who plays Daenerys Targaryen?")])
print([x.id for x in compression_retriever.invoke("Who plays Daenerys Targaryen?")])

['094f22df-9334-4e50-9469-cf1629adf240', '77e401bb-8aca-42f4-80f5-2a1ed6af3ce3', '74d9098c-0fc7-48d0-9699-76a6135e1631', '8a1ae1e8-f039-40ae-aa85-456d2cbb5f0d']
['74d9098c-0fc7-48d0-9699-76a6135e1631', '77e401bb-8aca-42f4-80f5-2a1ed6af3ce3', '8a1ae1e8-f039-40ae-aa85-456d2cbb5f0d']


In [ ]:
def invoke_reranking_rag_chain(question, retriever_type="basic"):
    if retriever_type == "basic":
        retrieved_docs = retriever.invoke(question)
    elif retriever_type == "compression":
        retrieved_docs = compression_retriever.invoke(question)
    elif retriever_type == "flash":
        # TODO: Implement Flash retriever

        # IMPORTANT: wont work because of versioning problems in FlashrankRank

        # your code here
        pass

    else:
        raise NotImplementedError("No other retrievers implemented yet.")
    for doc in retrieved_docs:
        display(Markdown(f"### {doc.metadata['source']}"))
        display(Markdown(doc.page_content))
    print("#" * 50)
    input_prompt = prompt.invoke({"question": question, "context": "\n\n".join(doc.page_content for doc in retrieved_docs)})
    answer = llm.invoke(input_prompt)
    return answer

In [ ]:
question = "Who plays Daenerys Targaryen?"
answer = invoke_reranking_rag_chain(question, "basic")
print("Answer:", answer)

### Game_of_Thrones.md

Across the Narrow Sea in Pentos, siblings Viserys Targaryen (Harry Lloyd) and Daenerys Targaryen
(Emilia Clarke) (colloquially referred to as "Dany") are in exile, with the former plotting to reclaim his
father's throne. Daenerys is forced into marrying Khal Drogo (Jason Momoa), a leader of the nomadic
Dothraki. Her retinue eventually comes to include the exiled knight Ser Jorah Mormont (Iain Glen), her
aide Missandei (Nathalie Emmanuel), mercenary Daario Naharis (Michiel Huisman), and elite soldier
[Grey Worm (Jacob Anderson).[[13]]](https://en.wikipedia.org/wiki/List_of_Game_of_Thrones_characters#Grey_Worm)

**Main cast and characters**

### Game_of_Thrones.md

-----

yj y ( )
Bolton (Michael McElhatton), and Roose's illegitimate son, Ramsay Snow (Iwan Rheon). Robb accepts
help from the healer Talisa Maegyr (Oona Chaplin), while elsewhere, Arya befriends blacksmith's
apprentice Gendry (Joe Dempsie) and assassin Jaqen H'ghar (Tom Wlaschiha). In the Stormlands, the tall
warrior Brienne of Tarth (Gwendoline Christie) is introduced to Catelyn.

In King's Landing, Ned's best friend, King Robert I Baratheon (Mark Addy), shares a loveless political
marriage with Cersei Lannister (Lena Headey). Her younger twin brother, Ser Jaime (Nikolaj Coster-
Waldau), serves on the Kingsguard while their younger brother Tyrion (Peter Dinklage) is attended by his
mistress Shae (Sibel Kekilli) and mercenary Bronn (Jerome Flynn). Cersei's father is Tywin (Charles)
Dance), head of House Lannister and the richest man in Westeros. Cersei has two sons: Joffrey (Jack
Gleeson) and Tommen (Dean-Charles Chapman). Joffrey is guarded by the scar-faced warrior Sandor
["The Hound" Clegane (Rory McCann).[[14]]](https://en.wikipedia.org/wiki/Sandor_Clegane)

The king's Small Council includes his treasurer, Petyr "Littlefinger" Baelish (Aidan Gillen), and his
spymaster, Varys (Conleth Hill). In Dragonstone, Robert's younger brother, Stannis (Stephen Dillane), is
advised by foreign priestess Melisandre (Carice van Houten) and former smuggler Ser Davos Seaworth
(Liam Cunningham). The Tyrell family from the Reach is represented at court by Margaery (Natalie
Dormer). The High Sparrow (Jonathan Pryce) is given power as a religious leader, while, in Dorne, the
[warrior Ellaria Sand (Indira Varma) seeks vengeance against the Lannisters.[[13]]](https://en.wikipedia.org/wiki/Ellaria_Sand)

### Game_of_Thrones.md

# Cast and characters

_Game of Thrones has an ensemble cast which has been_
estimated to be the largest on television.[[9]] In 2014,
several actors' contracts were renegotiated to include a
seventh-season option.[[10]] By the final season, five of
the main cast members made $1 million per episode,
making them among the highest paid television
[performers.[[11][12]]](https://en.wikipedia.org/wiki/List_of_highest_paid_American_television_stars)


**Related**


y

Frank Doelger

Bernadette Caulfield

Bryan Cogman

Miguel Sapochnik

David Nutter

**Producers** Mark Huffam

Joanna Burn

Chris Newman

Greg Spence

Lisa McAtackney

Duncan Muggoch


**Production**
**locations**


United Kingdom

Croatia

Iceland

Spain

Malta

Morocco

Canada


**Running time** 50–82 minutes


**Production**
**companies**


HBO Entertainment

Television 360

Grok! Television

Generator Entertainment

Startling Television

Bighead Littlehead


**Original release**


**Network** HBO

**Release** April 17, 2011 –
May 19, 2019


Eddard "Ned" Stark (Sean Bean) is the head of House _Thronecast_
Stark. His children with his wife, Catelyn (Michelle _After the Thrones_
Fairley), include: Robb (Richard Madden), Sansa

_House of the Dragon_

(Sophie Turner), Arya (Maisie Williams) and Bran

_A Knight of the Seven Kingdoms)_

(Isaac Hempstead-Wright). Ned also has an
illegitimate son, Jon Snow (Kit Harington), who, along)
with his scholarly friend, Samwell Tarly (John Bradley) serve in the Night's Watch under Lord
Commander Jeor Mormont (James Cosmo). The Wildlings living north of the Wall include Gilly (Hannah
[Murray) and the warriors Tormund Giantsbane (Kristofer Hivju) and Ygritte (Rose Leslie).[[13]]](https://en.wikipedia.org/wiki/Hannah_Murray)

### Game_of_Thrones.md

**_House of the Dragon_**

In September 2019, _Deadline Hollywood reported that a second prequel from Martin and Ryan Condal_
that "tracks the beginning of the end for House Targaryen" was close to receiving a pilot order from
HBO; the project is not considered an original sixth script, as it builds upon Cogman's idea from
2017.[[466]] This prequel, titled House of the Dragon, was commissioned as a complete series on October
29, 2019. The 10-episode series is to be based on material from Fire and Blood, executive produced by)
Martin, Vince Gerardis, Condal, and Miguel Sapochnik; the latter two are to be its showrunners as
well.[[467]] In January 2020, HBO stated that the series is scheduled for a 2022 release and that the writing
process has begun.[[468]] Casting for the series started in July 2020.[[469]] In October 2020, it was revealed
[that Paddy Considine was cast as King Viserys I Targaryen.[[470]] In December 2020, three more castings](https://en.wikipedia.org/wiki/Paddy_Considine)
were announced: Olivia Cooke as Alicent Hightower, Emma D'Arcy as Rhaenyra Targaryen, and Matt
[Smith as Daemon Targaryen.[[471]] In February 2021, HBO chief content officer Casey Bloys stated that](https://en.wikipedia.org/wiki/Matt_Smith)
the show would start production in April, with filming occurring in England.[[472][473]] In February 2021,
Steve Toussaint, Eve Best, Rhys Ifans, and Sonoya Mizuno were confirmed to also be starring in the
series.[[474]] In March 2022, HBO announced a series premiere date of August 21, 2022, followed by the
[release of the official teaser trailer.[[475]]](https://en.wikipedia.org/wiki/Teaser_(trailer))

**_A Knight of the Seven Kingdoms_**

##################################################
Answer:  Emilia Clarke plays Daenerys Targaryen in Game of Thrones.

Assistant: Emilia


In [ ]:
question = "Who plays Daenerys Targaryen?"
answer = invoke_reranking_rag_chain(question, "compression")
print("Answer:", answer)

### Game_of_Thrones.md

# Cast and characters

_Game of Thrones has an ensemble cast which has been_
estimated to be the largest on television.[[9]] In 2014,
several actors' contracts were renegotiated to include a
seventh-season option.[[10]] By the final season, five of
the main cast members made $1 million per episode,
making them among the highest paid television
[performers.[[11][12]]](https://en.wikipedia.org/wiki/List_of_highest_paid_American_television_stars)


**Related**


y

Frank Doelger

Bernadette Caulfield

Bryan Cogman

Miguel Sapochnik

David Nutter

**Producers** Mark Huffam

Joanna Burn

Chris Newman

Greg Spence

Lisa McAtackney

Duncan Muggoch


**Production**
**locations**


United Kingdom

Croatia

Iceland

Spain

Malta

Morocco

Canada


**Running time** 50–82 minutes


**Production**
**companies**


HBO Entertainment

Television 360

Grok! Television

Generator Entertainment

Startling Television

Bighead Littlehead


**Original release**


**Network** HBO

**Release** April 17, 2011 –
May 19, 2019


Eddard "Ned" Stark (Sean Bean) is the head of House _Thronecast_
Stark. His children with his wife, Catelyn (Michelle _After the Thrones_
Fairley), include: Robb (Richard Madden), Sansa

_House of the Dragon_

(Sophie Turner), Arya (Maisie Williams) and Bran

_A Knight of the Seven Kingdoms)_

(Isaac Hempstead-Wright). Ned also has an
illegitimate son, Jon Snow (Kit Harington), who, along)
with his scholarly friend, Samwell Tarly (John Bradley) serve in the Night's Watch under Lord
Commander Jeor Mormont (James Cosmo). The Wildlings living north of the Wall include Gilly (Hannah
[Murray) and the warriors Tormund Giantsbane (Kristofer Hivju) and Ygritte (Rose Leslie).[[13]]](https://en.wikipedia.org/wiki/Hannah_Murray)

### Game_of_Thrones.md

-----

yj y ( )
Bolton (Michael McElhatton), and Roose's illegitimate son, Ramsay Snow (Iwan Rheon). Robb accepts
help from the healer Talisa Maegyr (Oona Chaplin), while elsewhere, Arya befriends blacksmith's
apprentice Gendry (Joe Dempsie) and assassin Jaqen H'ghar (Tom Wlaschiha). In the Stormlands, the tall
warrior Brienne of Tarth (Gwendoline Christie) is introduced to Catelyn.

In King's Landing, Ned's best friend, King Robert I Baratheon (Mark Addy), shares a loveless political
marriage with Cersei Lannister (Lena Headey). Her younger twin brother, Ser Jaime (Nikolaj Coster-
Waldau), serves on the Kingsguard while their younger brother Tyrion (Peter Dinklage) is attended by his
mistress Shae (Sibel Kekilli) and mercenary Bronn (Jerome Flynn). Cersei's father is Tywin (Charles)
Dance), head of House Lannister and the richest man in Westeros. Cersei has two sons: Joffrey (Jack
Gleeson) and Tommen (Dean-Charles Chapman). Joffrey is guarded by the scar-faced warrior Sandor
["The Hound" Clegane (Rory McCann).[[14]]](https://en.wikipedia.org/wiki/Sandor_Clegane)

The king's Small Council includes his treasurer, Petyr "Littlefinger" Baelish (Aidan Gillen), and his
spymaster, Varys (Conleth Hill). In Dragonstone, Robert's younger brother, Stannis (Stephen Dillane), is
advised by foreign priestess Melisandre (Carice van Houten) and former smuggler Ser Davos Seaworth
(Liam Cunningham). The Tyrell family from the Reach is represented at court by Margaery (Natalie
Dormer). The High Sparrow (Jonathan Pryce) is given power as a religious leader, while, in Dorne, the
[warrior Ellaria Sand (Indira Varma) seeks vengeance against the Lannisters.[[13]]](https://en.wikipedia.org/wiki/Ellaria_Sand)

### Game_of_Thrones.md

**_House of the Dragon_**

In September 2019, _Deadline Hollywood reported that a second prequel from Martin and Ryan Condal_
that "tracks the beginning of the end for House Targaryen" was close to receiving a pilot order from
HBO; the project is not considered an original sixth script, as it builds upon Cogman's idea from
2017.[[466]] This prequel, titled House of the Dragon, was commissioned as a complete series on October
29, 2019. The 10-episode series is to be based on material from Fire and Blood, executive produced by)
Martin, Vince Gerardis, Condal, and Miguel Sapochnik; the latter two are to be its showrunners as
well.[[467]] In January 2020, HBO stated that the series is scheduled for a 2022 release and that the writing
process has begun.[[468]] Casting for the series started in July 2020.[[469]] In October 2020, it was revealed
[that Paddy Considine was cast as King Viserys I Targaryen.[[470]] In December 2020, three more castings](https://en.wikipedia.org/wiki/Paddy_Considine)
were announced: Olivia Cooke as Alicent Hightower, Emma D'Arcy as Rhaenyra Targaryen, and Matt
[Smith as Daemon Targaryen.[[471]] In February 2021, HBO chief content officer Casey Bloys stated that](https://en.wikipedia.org/wiki/Matt_Smith)
the show would start production in April, with filming occurring in England.[[472][473]] In February 2021,
Steve Toussaint, Eve Best, Rhys Ifans, and Sonoya Mizuno were confirmed to also be starring in the
series.[[474]] In March 2022, HBO announced a series premiere date of August 21, 2022, followed by the
[release of the official teaser trailer.[[475]]](https://en.wikipedia.org/wiki/Teaser_(trailer))

**_A Knight of the Seven Kingdoms_**

##################################################
Answer:  Daenerys Targaryen is played by Emilia Clarke. She portrays the character of Da


## Flash Retriever
Now try the FlashRank by yourself:
* [Damodaran, P. (2023). FlashRank GitHub](https://github.com/PrithivirajDamodaran/FlashRank)
* [LangChain Docs](https://python.langchain.com/docs/integrations/retrievers/flashrank-reranker/)

In [ ]:
# TODO: Implement Flash retriever

# your code here




In [ ]:
def invoke_reranking_rag_chain(question, retriever_type="basic"):
    if retriever_type == "basic":
        retrieved_docs = retriever.invoke(question)
    elif retriever_type == "compression":
        # TODO: implement compression retriever in here
        pass
    elif retriever_type == "flash":
        # TODO: Implement Flash retriever

        # IMPORTANT: wont work because of versioning problems in FlashrankRank

        # your code here
        pass

    else:
        raise NotImplementedError("No other retrievers implemented yet.")
    for doc in retrieved_docs:
        display(Markdown(f"### {doc.metadata['source']}"))
        display(Markdown(doc.page_content))
    print("#" * 50)
    input_prompt = prompt.invoke({"question": question, "context": "\n\n".join(doc.page_content for doc in retrieved_docs)})
    answer = llm.invoke(input_prompt)
    return answer